Blok 1: Kütüphaneler ve Dosya Yolları

Önce temel araçlarımızı çağıralım ve yolları tanımlayalım.

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
# Keras'ı doğrudan tf üzerinden çağıralım (Pylance bunu daha kolay anlar)
from tensorflow import keras
layers = keras.layers
models = keras.models
applications = keras.applications

import matplotlib.pyplot as plt
import sklearn.metrics as metrics # sklearn kuruluysa sarı çizgi gitmeli

# Dosya yolları
LIDAR_PATH = r"C:\Users\alkay\ITU_C\chactun\data\Chactun_ML_ready_lidar\lidar"
MASK_PATH = r"C:\Users\alkay\ITU_C\chactun\data\Chactun_ML_ready_masks\masks"

# Eğitim ve Test sınırları
TRAIN_END = 1764
TEST_END = 2093

Blok 2: Veri Yükleme Fonksiyonu

Bu blokta .tif dosyalarını okuyup, maskeleri birleştirip (3 kanallı hale getirip) normalleştireceğiz.

In [4]:
def load_data(start_idx, end_idx):
    images = []
    masks = []
    
    for i in range(start_idx, end_idx + 1):
        # LiDAR görüntüsünü oku
        img_path = os.path.join(LIDAR_PATH, f"tile_{i}_lidar.tif")
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (224, 224)) # VGG19 standart boyutu
        images.append(img / 255.0) # Normalizasyon

        # 3 Farklı maskeyi oku ve birleştir
        # Siyah = 0 (pozitif), Beyaz = 255 (negatif) olduğu için 1 - (mask/255) yapıyoruz
        m_aguada = cv2.imread(os.path.join(MASK_PATH, f"tile_{i}_mask_aguada.tif"), cv2.IMREAD_GRAYSCALE)
        m_building = cv2.imread(os.path.join(MASK_PATH, f"tile_{i}_mask_building.tif"), cv2.IMREAD_GRAYSCALE)
        m_platform = cv2.imread(os.path.join(MASK_PATH, f"tile_{i}_mask_platform.tif"), cv2.IMREAD_GRAYSCALE)
        
        # Maskeleri birleştirip 3 kanallı hale getir (Building, Aguada, Platform kanalları)
        combined_mask = np.stack([m_building, m_aguada, m_platform], axis=-1)
        combined_mask = cv2.resize(combined_mask, (224, 224))
        
        # İnvert işlemi: 0 olanlar 1 (pozitif), 255 olanlar 0 (negatif) olsun
        combined_mask = 1.0 - (combined_mask / 255.0)
        masks.append(combined_mask)
        
    return np.expand_dims(np.array(images), -1), np.array(masks)

# Verileri yükle (Bellek sorunu olursa parça parça yüklenebilir)
print("Eğitim verileri yükleniyor...")
X_train, y_train = load_data(0, TRAIN_END)
print("Test verileri yükleniyor...")
X_test, y_test = load_data(TRAIN_END + 1, TEST_END)

Eğitim verileri yükleniyor...
Test verileri yükleniyor...


In [3]:
import tensorflow as tf
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print(f"✅ GPU Algılandı: {gpu_devices}")
else:
    print("❌ GPU Algılanamadı, CPU kullanılacak.")

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

Blok 3: VGG-19 Tabanlı U-Net Modeli

VGG-19'un ilk katmanlarını "Encoder" (özellik çıkarıcı) olarak kullanacağız. Segmentasyon yaptığımız için görüntüyü tekrar eski boyutuna getiren bir "Decoder" eklememiz gerekiyor.

In [ ]:
def build_vgg19_unet(input_shape=(224, 224, 1)):
    # VGG19 tabanlı Encoder (Transfer Learning)
    # Tek kanallı LiDAR olduğu için giriş katmanını manuel ayarlıyoruz
    base_vgg = applications.VGG19(include_top=False, weights=None, input_shape=input_shape)
    
    # Encoder kısımlarını alalım (Skip connections için)
    s1 = base_vgg.get_layer("block1_conv2").output # 224x224
    s2 = base_vgg.get_layer("block2_conv2").output # 112x112
    s3 = base_vgg.get_layer("block3_conv4").output # 56x56
    s4 = base_vgg.get_layer("block4_conv4").output # 28x28
    
    # Bridge (En alt katman)
    bridge = base_vgg.get_layer("block5_conv4").output # 14x14
    
    # Decoder (Yükseltme Katmanları)
    def upsample_block(x, skip, filters):
        x = layers.UpSampling2D((2, 2))(x)
        x = layers.Concatenate()([x, skip])
        x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
        x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
        return x

    d1 = upsample_block(bridge, s4, 512)
    d2 = upsample_block(d1, s3, 256)
    d3 = upsample_block(d2, s2, 128)
    d4 = upsample_block(d3, s1, 64)
    
    # Çıkış Katmanı (3 Sınıf için Sigmoid - Çok etiketli sınıflandırma)
    outputs = layers.Conv2D(3, 1, activation="sigmoid")(d4)
    
    return models.Model(base_vgg.input, outputs)

model = build_vgg19_unet()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Blok 4: Eğitim (Training)

Modeli eğitmeye başlayalım. Arkeolojik verilerde nesneler seyrek olduğu için binary_crossentropy başlangıç için iyidir.

In [ ]:
history = model.fit(
    X_train, y_train, 
    validation_data=(X_test, y_test), 
    epochs=25, 
    batch_size=16
)

# Eğitim sonucunu görselleştir
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.legend()
plt.show()

### Analizler

1. Sayısal Analiz: IoU (Intersection over Union) Skoru
Segmentasyonun "altın standardı" IoU (Kesişim bölü Birleşim) değeridir. Bu, modelin tahmin ettiği maske ile gerçek maskenin ne kadar üst üste bindiğini ölçer.

In [ ]:
from sklearn.metrics import jaccard_score

def calculate_iou(model, X_test, y_test):
    y_pred = model.predict(X_test)
    # Tahminleri 0.5 eşiğiyle 0 veya 1'e çevirelim
    y_pred_thresholded = (y_pred > 0.5).astype(np.uint8)
    
    classes = ['Building', 'Aguada', 'Platform']
    ious = []
    
    print("\n--- SINIF BAZLI IoU SKORLARI ---")
    for i, class_name in enumerate(classes):
        # Her sınıf için IoU hesapla
        iou = jaccard_score(y_test[..., i].flatten(), y_pred_thresholded[..., i].flatten(), pos_label=1)
        ious.append(iou)
        print(f"{class_name} IoU: {iou:.4f}")
    
    print(f"\nOrtalama (Mean) IoU: {np.mean(ious):.4f}")

# Modeli test verisiyle değerlendir
calculate_iou(model, X_test, y_test)

2. Görselleştirme: Tahminleri Karşılaştırma
Modelin neyi doğru, neyi yanlış bildiğini görmenin en iyi yolu; Orijinal LiDAR, Gerçek Maske ve Modelin Tahminini yan yana koymaktır.

In [ ]:
def visualize_results(model, X_test, y_test, num_samples=3):
    # Rastgele örnekler seç
    indices = np.random.randint(0, len(X_test), num_samples)
    y_pred = model.predict(X_test[indices])
    y_pred_thresholded = (y_pred > 0.5).astype(np.uint8)

    plt.figure(figsize=(15, 5 * num_samples))
    
    for i, idx in enumerate(indices):
        # 1. Sütun: Orijinal LiDAR
        plt.subplot(num_samples, 3, i*3 + 1)
        plt.imshow(X_test[idx].squeeze(), cmap='gray')
        plt.title(f"Örnek {idx}: Orijinal LiDAR")
        plt.axis('off')

        # 2. Sütun: Gerçek Maske (Ground Truth)
        # Sınıfları renkli görmek için 3 kanalı da gösteriyoruz
        plt.subplot(num_samples, 3, i*3 + 2)
        plt.imshow(y_test[idx])
        plt.title("Gerçek Maske")
        plt.axis('off')

        # 3. Sütun: Modelin Tahmini
        plt.subplot(num_samples, 3, i*3 + 3)
        plt.imshow(y_pred_thresholded[i])
        plt.title("Model Tahmini")
        plt.axis('off')

    plt.tight_layout()
    plt.show()

# Tahminleri görselleştir
visualize_results(model, X_test, y_test)

3. Eğitim Analizi: Loss ve Accuracy Grafikleri
Eğitim sırasında modelin "ezberleyip ezberlemediğini" (overfitting) anlamak için bu grafikleri çizdirmeliyiz.

In [ ]:
def plot_history(history):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Loss Grafiği
    ax1.plot(history.history['loss'], label='Train Loss')
    ax1.plot(history.history['val_loss'], label='Val Loss')
    ax1.set_title('Model Kaybı (Loss)')
    ax1.set_xlabel('Epoch')
    ax1.legend()

    # Accuracy Grafiği
    ax2.plot(history.history['accuracy'], label='Train Acc')
    ax2.plot(history.history['val_accuracy'], label='Val Acc')
    ax2.set_title('Model Doğruluğu (Accuracy)')
    ax2.set_xlabel('Epoch')
    ax2.legend()

    plt.show()

plot_history(history)